## Deployment in azure for the fastText Model

In [1]:
import azureml.core
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.34.0


### Create the model path

In [3]:
model_path='./model'

### Create the workspace

In [5]:
from azureml.core import Workspace
ws = Workspace.create(name='myworkspace',
               subscription_id='d2a3b3d7-84df-4290-944a-a1f53aeea99c',
               resource_group='Ope711',
               location='West Europe')

Deploying AppInsights with name myworkspinsights3bc6a205.
Deployed AppInsights with name myworkspinsights3bc6a205. Took 3.9 seconds.
Deploying StorageAccount with name myworkspstorageac86dc00d.
Deploying KeyVault with name myworkspkeyvault59b6a437.
Deployed StorageAccount with name myworkspstorageac86dc00d. Took 23.98 seconds.
Deploying Workspace with name myworkspace.
Deployed KeyVault with name myworkspkeyvault59b6a437. Took 31.51 seconds.
Deployed Workspace with name myworkspace. Took 21.8 seconds.


In [6]:
from azureml.core import Workspace
ws = Workspace(subscription_id='d2a3b3d7-84df-4290-944a-a1f53aeea99c',
               resource_group="Ope711",
               workspace_name="myworkspace")

### Register the model in Azure

In [7]:
from azureml.core.model import Model

# Register model
model = Model.register(ws, model_name="sentitweet", model_path=model_path)

Registering model sentitweet


In [8]:
print(model.name, model.id, model.version, sep='\t')

sentitweet	sentitweet:1	1


In [9]:
model_root = Model.get_model_path("sentitweet", version = 1, _workspace=ws)

In [10]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

#### Write the score file

In [43]:
%%writefile score.py
import tensorflow as tf
import json
import joblib
import numpy as np
from azureml.core.model import Model
from azureml.core import Workspace
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import pickle  
import nltk


# Called when the service is loaded
def init():
    global tokenizer
    global modele
    # load model
    model_root = Model.get_model_path('sentitweet')
    modele=load_model(os.path.join(model_root,'model.h5'))
    # tokenizer
    token_file = open(os.path.join(model_root, 'tokenizer.pickle'), 'rb')
    tokenizer = pickle.load(token_file)
    
# Called when a request is received
def run(raw_data):
    print(raw_data)
    

    tokeniz = nltk.RegexpTokenizer(r'\w+')
    raw_data = tokeniz.tokenize(raw_data)

    # Tokenize texte
    x_test = pad_sequences(tokenizer.texts_to_sequences([raw_data]), maxlen=30)
    # Prediction
    score = modele.predict([x_test])[0]
    # Sentiment
    if score <= 0.5: 
        sentiment = "negatif"
    elif score> 0.5: 
        sentiment = "positif"
    return {"score": float(score),"sentiment": sentiment}  

Overwriting score.py


### Create the environment and the dependencies

In [44]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

env = Environment(name='myenv')
python_packages = ['nltk', 'numpy','tensorflow']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

In [45]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "tweets",  "method" : "FTX"}, 
                                               description='Sentiment detection of tweets')

In [46]:
inference_config = InferenceConfig(environment=env, entry_script='./score.py')

### Deploy the model in Azure

In [47]:
service = Model.deploy(
    ws,
    "hiver",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-21 20:12:36+02:00 Creating Container Registry if not exists.
2021-09-21 20:12:37+02:00 Registering the environment.
2021-09-21 20:12:41+02:00 Use the existing image.
2021-09-21 20:12:41+02:00 Generating deployment configuration..
2021-09-21 20:12:43+02:00 Submitting deployment to compute.
2021-09-21 20:12:46+02:00 Checking the status of deployment hiver..
2021-09-21 20:12:56+02:00 Checking the status of inference endpoint hiver.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [48]:
service.get_logs()

'2021-09-21T18:12:49,640315857+00:00 - rsyslog/run \n2021-09-21T18:12:49,643913373+00:00 - nginx/run \n2021-09-21T18:12:49,642923769+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2021-09-21T18:12:49,645308779+00:00 - iot-server/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-09-21T18:12:49,852829370+00:00 - iot-server/finish 1 0\n2021-09-21T18:12:49,854277676+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (12)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 41\n2021-09-21 18:12:50.815630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library \'libcudart.so.11.0\'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_bf9470cf9708630f26b57013e1a0ad56/lib:/azureml-envs/azureml_bf9470cf9708

In [49]:
print(service.scoring_uri)

http://a7f7b42e-9216-48db-8ba5-8809f6f79fdc.westeurope.azurecontainer.io/score


### We test the deployed model

In [50]:
import requests
import json

uri = service.scoring_uri
requests.get("http://a7f7b42e-9216-48db-8ba5-8809f6f79fdc.westeurope.azurecontainer.io/score")
headers = {"Content-Type": "application/json"}
raw_data = {
    "query":"I love chocolate",
}
data = json.dumps(raw_data)
response = requests.post(uri, data=raw_data, headers=headers)
print(response.json())

{'score': 0.6391505002975464, 'sentiment': 'positif'}


In [52]:
import requests
import json

uri = service.scoring_uri
requests.get("http://a7f7b42e-9216-48db-8ba5-8809f6f79fdc.westeurope.azurecontainer.io/score")
headers = {"Content-Type": "application/json"}
raw_data = {
    "query":"The service was below average and the chips were terribly cold",
}
data = json.dumps(raw_data)
response = requests.post(uri, data=raw_data, headers=headers)
print(response.json())

{'score': 0.3435966372489929, 'sentiment': 'negatif'}
